In [ ]:
!pip install accelerate bitsandbytes

In [ ]:
import bitsandbytes, accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mistral-7B-Instruct-v0.2"
device = "cuda"

tokenizer = AutoTokenizer.from_pretrained(model_id)
if (tokenizer.pad_token is None):
    tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map="auto",
                                             load_in_8bit=True)

In [ ]:
def chat(save):
    save = save.split()
    i = len(save) - 1
    a1 = True
    while(a1):
        if(save[i] == '[/INST]'):
            for x in range(i):
                save.pop(0)
            save.pop(0)
            a1 = False
        else:
            i += -1
    save = ' '.join(save)
    return save
text = []
a1 = True
print("To exit type (Y)")
while a1:
    message = input("User: ")
    if(message == 'Y'):
        a1 = False
    else:
        text.append({"role":"user","content":message})
        inputs = tokenizer.apply_chat_template(text,return_tensors="pt",
                                               tokenize=True,
                                               add_generation_prompt=True).to(device)
        generation_kwargs = {"repetition_penalty": 1.3, "max_new_tokens": 50}
        outputs = model.generate(inputs,**generation_kwargs,
                                 pad_token_id=tokenizer.pad_token_id)
        save = tokenizer.decode(outputs[0])
        save1 = chat(save)
        text.append({"role":"assistant","content":f" {save1}"})
        print(f"Assistant: {save1}")